In [1]:
import numpy as np
import math
import scipy.stats as scps

Quadrature points

In [2]:
def quadrature(n, lbnd, ubnd):
    
    x1 = lbnd
    x2 = ubnd
    x = np.zeros(n)
    w = x
    EPS = 3e-14
    m = int(round((n+EPS)/2)) # flor function in matlab, rounding to the lower integer
    xm = (x2+x1)/2
    xl = (x2-x1)/2
    z1 = 1e99

    x = np.full(n+1, np.nan)
    w = np.full(n+1, np.nan)

    i = 1

    while i <= m:

        z = math.cos(math.pi*(i - 0.25)/(n + 0.5))

        while abs(z - z1) > EPS:
            p1 = 1
            p2 = 0
            j = 1

            while j <= n:
                p3 = p2
                p2 = p1
                p1 = ((2*j -1)*z*p2 - (j-1)*p3)/j
                j += 1

            pp = n*(z*p1 - p2)/(z*z - 1)
            z1 = z
            z = z1 - p1/pp

        x[i] = xm - xl*z
        x[n + 1 - i] = xm + xl*z
        w[i] = 2*xl/((1-z*z)*pp*pp)
        w[n + 1 - i] = w[i]
        i += 1

    x = x[1:]
    w = w[1:]

    return x, w

Minimal
=======

In [ ]:
EXPN = 10 # number of quadrature points
MMAX = 10 # maximum savings
NM = 100 # number of EGM gridpoints
TBAR = 25 # number of periods
SIGMA = 0.25 # sigma of normal distribution
Y = 1 # period income
R = 0.05 # interest rate
DF = 0.95 # discount factor

step = MMAX/NM

In [ ]:
# create quadrature nodes and weights
quadp, quadw = quadrature(EXPN, 0, 1)

# process through inverse normal distribution
quadstnorm = scps.norm.ppf(quadp)

# fix exogenous grod points on savings
savingsgrid = np.linspace(0, MMAX, NM)

In [ ]:
policy = np.full((NM + 1, 2, TBAR - 1), np.nan)
policy[0, :, :] = 0.00

In [ ]:
# Backward induction

for period in reversed(range(TBAR - 1)): # starts at 23, end at zero

    # obtain next periods wealth at every exog. savings point (A_{t+1}) using budget constraint
    w1 = Y + np.multiply((np.exp(quadstnorm*SIGMA)*(1+R)).reshape(10,1), savingsgrid) # M_{t+1}, dim (EXPN, NM)

    # every wealth level in t+1 corresponds to an optimal consumption level
    # since wealth points might be off grid, get optimal consumption using extrapolation
    if period == TBAR - 2:
        c1 = w1 # separate handling of last period
    else:
        # interpolate
        c1 = np.interp(w1, policy[:, 0, period + 1], policy[:, 1, period+1]) # c_{t+1}
        # extrapolate
        slope = (policy[-2, 1, period + 1] - policy[-1, 1, period + 1])/(policy[-2, 0, period + 1] - policy[-1, 0, period + 1])
        intercept = policy[-1, 1, period + 1] - policy[-1, 0, period + 1]*slope
        c1[c1 == np.max(policy[:, 1, period+1])] = intercept + slope*w1[c1 == np.max(policy[:, 1, period+1])]

    # obtain current period optimal consumption and wealth via Euler equation
    # since Euler eq. involves c_{t+1} above computations were necessary
    rhs = np.dot(quadw.T, 1/c1)
    policy[1:, 1, period] = 1/(DF*(1+R)*rhs)
    policy[1:, 0, period] = savingsgrid + policy[1:, 1, period]

Retirement
==========

Common elements
----------------------

In [3]:
# Model parameters

Tbar = 25 # number of periods (fist period is t=1) 
ngridm = 500 # number of grid points over assets
mmax = 50 # maximum level of assets
expn = 5 # number of quadrature points used in calculation of expectations
nsims = 10 # number of simulations
init = [10, 30] # interval of the initial wealth
r = 0.05 # interest rate
df = 0.95 # discount factor
sigma = 0.25 # sigma parameter in income shocks
duw = 0.35 #disutility of work
theta = 1.95 # CRRA coefficient (log utility if ==1)
inc0 = 0.75 # income equation: constant
inc1 = 0.04 # income equation: age coef
inc2 = 0.0002 # income equation: age^2 coef
cfloor =0.001 # consumption floor (safety net in retirement)
lambda_ = 0.02 # scale of the EV taste shocks 

In [4]:
# Functions: utility and budget constraint

def util(consumption, working):
    """CRRA utility"""
    
    u = (consumption**(1-theta)-1)/(1-theta)
    u = u - duw*(working)
    
    return u

def mutil(consumption):
    """Marginal utility CRRA"""
    
    mu = consumption**(-theta)
    
    return mu

def imutil(mutil):
    """Inverse marginal utility CRRA
    Consumption as a function of marginal utility"""
    
    cons = mutil**(-1/theta)
    
    return cons


def income(it, shock):
    """Income in period it given normal shock"""
    
    age = it + 20 # matlab strats counting at 1, Python at zero
    w = np.exp(inc0 + inc1*age - inc2*age**2 + shock)
    
    return w


def budget(it, savings, shocks, working):
    """Wealth, M_{t+1} in period t+1, where it == t
    
    Arguments
    ---------
        savings: np.array of savings with length ngridm
        shocks: np.array of shocks with length expn
    
    Returns
    -------
        w1: matrix with dimension (expn, ngridm) of all possible
    next period wealths
    """
    
    w1 = np.full((ngridm, expn), income(it + 1, shocks)*working).T + np.full((expn, ngridm), savings*(1+r))
    
    return w1

def mbudget():
    """Marginal budget:
    Derivative of budget with respect to savings"""
    
    mw1 = np.full((expn, ngridm), (1+r))
    
    return mw1

In [5]:
# Value function for worker

def value_function(working, it, x):
    
    # Remove zero column
    x = x.flatten('F')
    
    res = np.full(x.shape, np.nan)
    
    # Mark constraint region
    if it == Tbar - 1: # special handling of last period
        mask = x < (x+1) # all points in constraint region, rewrite in better way
    else:
        mask = x < value[1, 1, it] # credit constraint between 1st (zero) and second point
    
    res[mask] = util(x[mask], working) + df*value[0, working, it] #reshaped to array with length 2495
    
    ## To Do: Add interpolation for non credit constrained regions
    # becomes relevant for second to last period
    
    return res

In [6]:
# Value function for retiree

def vfcalc(it, x):
    """Value function calculation for the case of a worker"""
    
    vfres = np.full(x.shape, np.nan)
    
    # Mark constraint region
    if it == Tbar - 1: 
        mask = x < (x+1) 
    else:
        mask = x < value[1, 0, it] # different notation to above
    
    vfres[mask] = util(x[mask], 0) + df*value[0, 1, it]
    
    return vfres
    
    # TO DO: Add interpolation
    # becomes relevant after second to last period

In [7]:
# Calculation of expected value function

def chpr(x):
    
    mx = np.amax(x, axis = 0)
    mxx = x - mx
    res = np.exp(mxx[1, :]/lambda_)/np.sum(np.exp(mxx/lambda_), axis = 0)
    
    return res

In [8]:
# Calculation of probability to choose work, if a worker today

def logsum(x):
    
    mx = np.amax(x, axis = 0)
    mxx = x - mx
    res = mx + lambda_*np.log(np.sum(np.exp(mxx/lambda_), axis = 0))
    
    return res

In [9]:
# Interpolation

def interpolate(x, policy):
    
    xx = x.flatten('F')
    res = np.full((policy.shape[1], ngridm * expn), np.nan)
    
    res_ret = np.interp(xx, savingsgrid, policy[:, 0, period+1])
    res_ret[res_ret == np.max(savingsgrid)] = xx[res_ret == np.max(savingsgrid)]
    
    res_work = np.interp(xx, savingsgrid, policy[:, 0, period+1])
    res_work[res_work == np.max(savingsgrid)] = xx[res_work == np.max(savingsgrid)]
    
    res[0, :] = res_ret
    res[1, :] = res_work
    
    return res

Solve EGM
-------------

Current assumption that this is the solution to the workers problem.

Code for second to last period.
Runs, but incomplete and definitely have to revisit some parts concerning shape of objects and interpolate/extrapolate part.

In [ ]:
# Initialize grids
quadp, quadw = quadrature(expn,0,1)
quadstnorm = scps.norm.ppf(quadp)
savingsgrid = np.linspace(0, mmax, ngridm)

In [ ]:
# Initialize containers

# Container for endogenous gridpoints of (beginning-of-period) assets
# and corresponding consumption
policy = np.full((ngridm, 2, Tbar), np.nan)

# Value functions
value = np.full((ngridm, 2, Tbar), np.nan)

# Choice probabilities
pr_work = np.full((ngridm, Tbar), np.nan)

In [ ]:
# Handling of last period
endog_grid[:, Tbar - 1] = savingsgrid

policy[:, 0, Tbar-1] = savingsgrid
policy[:, 1, Tbar-1] = policy[:, 0, Tbar-1]

value[1:, 0, Tbar-1] = util(policy[1:, 0, Tbar-1], 0)
value[1:, 1, Tbar-1] = util(policy[1:, 0, Tbar-1], 1)
value[0, :, Tbar -1] = 0.00

pr_work[:, Tbar-1] = 1/(1 + np.exp(duw/sigma)) # from paper p.338, line 4 # actually not used!!!!

In [ ]:
# Start backward induction from second to last period

# for period in reversed(range(TBAR - 1)): # starts at 23, end at zero, i.e., 24 iterations
    
#     for choice in range[0, 1]:

period = 23
choice = 1
    
# M_{t+1}
wk1 = budget(period, savingsgrid, quadstnorm*sigma, choice)
wk1[wk1 < cfloor] = cfloor

# Value function
vl1 = np.full((2, ngridm * expn), np.nan)
vl1[1, :] = value_function(1, period + 1, wk1) # next period value of working
vl1[0, :] = value_function(0, period + 1, wk1) # next period value of retiring

# Choice probability working
# calculated only for choice == 1, i.e., worker
if choice == 1:
    pr1 = chpr(vl1)

# Next period consumption based on interpolation and extrapolation
# given grid points and associated consumption
cons1 = interpolate(wk1, policy)

# Marginal utility of consumption in next period
mu1 = pr1*mutil(cons1[1, :]) + (1 - pr1)*mutil(cons1[0, :])

# Marginal budget
# Note: Constant for this model formulation
mwk1 = mbudget()

# RHS of Euler eq., p 337, integrate out error of y
rhs = np.dot(quadw.T, np.multiply(mu1.reshape(wk1.shape, order = 'F'), mwk1))

####################
#Current period vars
####################

# Current period consumption from Euler equation
cons0 = imutil(df*rhs)

# Update containers related to consumption
policy[:, 0, period] = cons0
endog_grid[:, period] = savingsgrid + cons0
## TO DO!!!: Revisit policy object

# Update containers related to value functions
ev = np.dot(quadw.T, logsum(vl1).reshape(wk1.shape, order = 'F'))

## TO DO: Add line for choice == 0, i.e., retirement

Solve retiree
=============

Solution of the retirees problem.

Backward induction does not depend on the value function here since because no choice is made. Retirement is an absorbing state. => Backward induction works now!

But!!!: We need the value function value of retirement for the simulation: agents would be making choices by comparing the value function values of working and retirement => figure a way out how to calculate value function values! Potentially outsside the loop!

Questions:

Why does the Gaussian quadrature enter the retirees problem? There is no income involved, and I thought, quadrature is there to integrate out the shocks/errors in the income process...

In [10]:
# Initialize grids
quadp, quadw = quadrature(expn,0,1)
quadstnorm = scps.norm.ppf(quadp)
savingsgrid = np.linspace(0, mmax, ngridm)

In [14]:
# Initialize containers

# Container for endogenous gridpoints of (beginning-of-period) assets
# and corresponding consumption
policy = np.full((ngridm + 1, 2, Tbar), np.nan) # dim 0 are x = endog. grid, y are assoc cons values

# # Value functions
# value = np.full((ngridm, 2, Tbar), np.nan) # same dimension meaning as above


In [16]:
# Handling of last period
policy[1:, 0, Tbar-1] = savingsgrid
policy[1:, 1, Tbar-1] = policy[1:, 0, Tbar-1]

# value[:, 0, Tbar-1] = savingsgrid
# value[1:, 1, Tbar-1] = util(policy[1:, 0, Tbar-1], 1)
# value[0, 1, Tbar -1] = np.nan

In [17]:
# Ensuring that policy starts with a zero both for x and y for all period
policy[0, :, :] = 0.00

In [21]:
# Backward induction

for period in reversed(range(Tbar - 1)): # starts at 23, end at zero

    # M_{t+1}
    wk1 = budget(period, savingsgrid, quadstnorm*sigma, choice)

    # Next period consumption based on interpolation and extrapolation
    # given grid points and associated consumption
    cons1 = np.interp(wk1, policy[:, 0, period + 1], policy[:, 1, period+1])
    # extrapolate linearly right of max grid point
    slope = (policy[-2, 1, period + 1] - policy[-1, 1, period + 1])/(policy[-2, 0, period + 1] - policy[-1, 0, period + 1])
    intercept = policy[-1, 1, period + 1] - policy[-1, 0, period + 1]*slope
    cons1[cons1 == np.max(policy[:, 1, period+1])] = intercept + slope*wk1[cons1 == np.max(policy[:, 1, period+1])]
    # Consumption floor
    cons1[cons1 < cfloor] = cfloor

    # Marginal budget
    # Note: Constant for this model formulation
    mwk1 = mbudget()

    # RHS of Euler, integrate out error of y
    rhs = np.dot(quadw.T, np.multiply(mutil(cons1), mwk1))

    ####################
    #Current period vars
    ####################

    # Current period consumption from Euler equation
    cons0 = imutil(df*rhs)

    # Update containers related to consumption
    policy[1:, 0, period] = savingsgrid + cons0 # save x, endog grid M_{t+1}
    policy[1:, 1, period] = cons0 # save y, corresponding consumption
